In [ ]:
#| code-summary: Import module / Set options and theme
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import xml.etree.ElementTree as ET
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import ttest_rel
from statsmodels.stats.weightstats import ttest_ind
import numpy as np
import pingouin as pg
from scipy.stats import zscore
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots
import warnings
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 10)

In [ ]:
#| code-summary: Import cleaned data

df = pd.read_csv('./data/cleaned_df.csv')
df['Location_ID'] = df.groupby(['long', 'lat']).ngroup() + 1

# Manipulate Dataset

## Average Scenarios

The Average Scenarios dataset averages all the numerical columns of the scenarios into one row, outputing one row for each Location, Year, and RCP. This dataset is used when conducting EDA and visualizing overtrend 


In [ ]:
#| code-summary: Clean data (Average Scenarios)

group_list = ['Park', 'long', 'lat', 'veg', 'year', 'TimePeriod', 'RCP','treecanopy', 'Ann_Herb', 'Bare', 'Herb', 'Litter', 'Shrub', 'El', 'Sa','Cl', 'RF', 'Slope', 'E', 'S']
veg_location = df.drop(labels='scenario',axis=1).groupby(group_list).mean().reset_index()
# veg_location['T_Annual'] = (veg_location['T_Annual'] - veg_location['T_Annual'].min()) / (veg_location['T_Annual'].max() - veg_location['T_Annual'].min())

# Convert to numeric, coercing errors to NaN
numeric_series = pd.to_numeric(veg_location['RCP'], errors='coerce')

numeric_series

# Fill NaNs with original non-numeric values
veg_location['RCP'] = numeric_series.fillna(veg_location['RCP'])

four = veg_location[veg_location['RCP'].isin([4.5])]
eight = veg_location[veg_location['RCP'].isin([8.5])]
four_h = veg_location[veg_location['RCP'].isin(['historical'])]
four_h['RCP'] = 4.5
eight_h = veg_location[veg_location['RCP'].isin(['historical'])]
eight_h['RCP'] = 8.5

df_con = pd.concat([four_h, four, eight_h, eight], ignore_index=True)
df_con['Location_ID'] = df_con.groupby(['long', 'lat']).ngroup() + 1

df_con.head(5)

## All Scenarios


In [ ]:
# Convert to numeric, coercing errors to NaN
numeric_series = pd.to_numeric(df['RCP'], errors='coerce')

numeric_series

# Fill NaNs with original non-numeric values
df['RCP'] = numeric_series.fillna(df['RCP'])

four = df[df['RCP'].isin([4.5])]
eight = df[df['RCP'].isin([8.5])]
four_h = df[df['RCP'].isin(['historical'])]
four_h['RCP'] = 4.5
eight_h = df[df['RCP'].isin(['historical'])]
eight_h['RCP'] = 8.5

df_orig = pd.concat([four_h, four, eight_h, eight], ignore_index=True)
df_orig['Location_ID'] = df_orig.groupby(['long', 'lat']).ngroup() + 1

df_orig.head(5)

# Data Exploration

## Basic Statistics

**Basic Statistics**<br>
 - 79 years of prediction (2021~2099)<br>
 - 40 scenarios (sc22~sc61)<br>
 - 2 RCP scenarios(4.5, 8.5)<br>
 - 113 locations<br>

**Explanation**<br>
 - The data is collected over 113 locations within the Natural Bridge National Monument. (Number of Unique latitude, longitude combinations)<br>
 - This dataset is composed of 41 years of historical data and 79 years worth of predictions. Since there can be only one scenario for past data, all historical data is labeled as 'sc1' or scenario one<br>
 - From the predicted years (2021 to 2099), There are two RCP scenarios which changes only the RCP variable and 40 scenarios which simulate 86 other variables.<br>


Based on each combination of scenarios, a prediction is made for each location point regarding various columns of the dataset including annual and seasonal percipitation, seasonal dry soil days, seasonal evaporation, seasonal extreme short term dry stress, soil water availability to output a **final prediction for Annual and seasonal temperatures.**

**What is RCP?**

Representative Concentration Pathways : A group of scenarios where CO2 emmission is predicted like the image below<br>
 - *The dataset consists of two RCP scenarios 4.5 and 8.5*

![](./images/RCP.png){width=400}

source : Representative Concentration Pathway. (2024, May 2). In Wikipedia. https://en.wikipedia.org/wiki/Representative_Concentration_Pathway

## Location
Where is this data located and how does it look like?

The data points were sampled at the Natural Bridge National Monument in Utah. And for a better idea of The plots below show two different location aspects of the dataset. The first plot is the average annual temperature for each location point in the year 2099. Since the temperature for predictions increase over time, the last year for the dataset was chosen for a more dramatic comparison

The second plot is a scatter plot of the locations of vegetations. By comparing the two graphs, we can tell that there isn't much correlation with vegetation and annual temperature but rather a correlation with the location(latitude, longitude) and temperature. We will get to this in the following visualizations. 


In [ ]:
#| code-summary: Map Visualizations

map = df_con[df_con['year']==2099].groupby(['long','lat'])['T_Annual'].mean().reset_index()

filtered_df = map
fig = px.scatter_mapbox(filtered_df, lat="lat", lon="long", color="T_Annual", size="T_Annual",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=8, zoom=11,
                  mapbox_style="open-street-map")

fig.update_layout(
    title={
        'text': "<b>Average Temperature (2099) </b>",
        'y': 0.97,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    margin={"r": 0, "t": 40, "l": 0, "b": 0}
    )

fig.show()

map = df_con[df_con['year']==2099].groupby(['long','lat','veg']).size().reset_index()

filtered_df = map

# Create the scatter mapbox
fig = px.scatter_mapbox(map, lat="lat", lon="long", color="veg",
                        color_continuous_scale=px.colors.cyclical.IceFire, size_max=8, zoom=11,
                        mapbox_style="open-street-map")

# Update the layout with the new legend title and position
fig.update_layout(
    title={
        'text': "<b>Vegetation Location</b>",
        'y': 0.97,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    coloraxis_colorbar={
        'title': 'Vegetation Level'  # Change this to your desired legend title
    },
    legend={
        'x': 1,  # Position the legend to the right
        'y': 0.8,  # Center the legend vertically
        'xanchor': 'left',  # Anchor the legend's x position to the left side
        'yanchor': 'middle'  # Anchor the legend's y position to the middle
    },
    margin={"r": 0, "t": 40, "l": 0, "b": 0}
)
fig.update_traces(marker=dict(size=10))  # Set the desired fixed marker size

# Show the figure
fig.show()

## Temperature/Percipitation Trends

The following plots were drawn by average all scenarios, locations, and RCPs for a given year for annual temperature and annual percipitation to see the overall trend of the predictions of the dataset. Predictions were made from the year 2021 which is 

We can see that the predictions portray an increase in temperature but a fluctuation with percipitation allowing us to make an educated guess that temperature is the more important variable for RCP scenarios which deal with CO2 emission.


In [ ]:
#| code-summary: Temperature / Percipitation Predictions Overview

# Assuming 'veg_location' is your DataFrame
# Filter the DataFrame for 'RCP' values 'historical' and 4.5
filtered_data = df_con.groupby(['year'])['T_Annual'].mean().reset_index()

# Create the line plot
fig = px.line(
    data_frame=filtered_data,
    x='year',
    y='T_Annual',
    title='<b>Annual Temperature</b>',
    labels={'T_Annual': 'Annual Temperature'},
    line_shape='spline'
)

# Add a vertical line at year 2021
fig.add_shape(
    dict(
        type='line',
        x0=2021,
        y0=filtered_data['T_Annual'].min()/1.1,
        x1=2021,
        y1=filtered_data['T_Annual'].max()*1.1,
        line=dict(
            color="Red",
            width=2,
            dash="dash",
        ),
    )
)

fig.add_annotation(
    dict(
        x=2021,  # Position the text to the right of the line
        y=filtered_data['T_Annual'].max(),  # Position the text at the middle of the y-axis
        xref="x",
        yref="y",
        text="Prediction",
        showarrow=False,
        font=dict(
            size=12,
            color="Red"
        ),
        align="center",
        xanchor="left"
    )
)

fig.update_layout(title={'x':0.5})
# Show the plot
fig.show()


# Assuming 'veg_location' is your DataFrame
# Filter the DataFrame for 'RCP' values 'historical' and 4.5
filtered_data = df_con.groupby(['year'])['PPT_Annual'].mean().reset_index()

# Create the line plot
fig = px.line(
    data_frame=filtered_data,
    x='year',
    y='PPT_Annual',
    title='<b>Annual Precipitation</b>',
    labels={'T_Annual': 'Annual Temperature'},
    line_shape='spline'
)

# Add a vertical line at year 2021
fig.add_shape(
    dict(
        type='line',
        x0=2021,
        y0=filtered_data['PPT_Annual'].min()/1.1,
        x1=2021,
        y1=filtered_data['PPT_Annual'].max()*1.1,
        line=dict(
            color="Red",
            width=2,
            dash="dash",
        ),
    )
)

fig.add_annotation(
    dict(
        x=2021,  # Position the text to the right of the line
        y=filtered_data['PPT_Annual'].max(),  # Position the text at the middle of the y-axis
        xref="x",
        yref="y",
        text="Prediction",
        showarrow=False,
        font=dict(
            size=12,
            color="Red"
        ),
        align="center",
        xanchor="left"
    )
)

fig.update_layout(title={'x':0.5})
# Show the plot
fig.show()

# Perspectives to Consider
What are some aspects of the datasets we can slice and dice or drill down to compare and retrieve meaningful insights?

Most numerical features are generated based on the scenario of the model that generated future data, and some numerical features such ase S,E,Slope, RF, Cl, Sa, El, treecanopy etc. are features that are fixed according to a unique location. Therefore categorical variables are the aspects of the datasets we can compare to create new insights

Categorical Variables : 
 - RCP
 - Vegetation
 - Scenario

 The following plots compare the predicted annual temperature for each category for the three categorical variables


In [ ]:
#| code-summary: Temperature RCP comparison

# Assuming 'veg_location' is your DataFrame
# Filter the DataFrame for 'RCP' values 'historical' and 4.5
filtered_data = df_con.groupby(['year','RCP'])['T_Annual'].mean().reset_index()

# Create the line plot
fig = px.line(
    data_frame=filtered_data,
    x='year',
    y='T_Annual',
    color='RCP',  # This will create lines for each unique value in 'veg' and color them differently
    title='<b>Annual Temperature by RCP Type</b>',
    labels={'T_Annual': 'Annual Temperature'},
    line_shape='spline'
)
fig.update_layout(title={'x':0.5})

# Add a vertical line at year 2021
fig.add_shape(
    dict(
        type='line',
        x0=2021,
        y0=filtered_data['T_Annual'].min()/1.1,
        x1=2021,
        y1=filtered_data['T_Annual'].max()*1.1,
        line=dict(
            color="Red",
            width=2,
            dash="dash",
        ),
    )
)

fig.add_annotation(
    dict(
        x=2021,  # Position the text to the right of the line
        y=filtered_data['T_Annual'].max(),  # Position the text at the middle of the y-axis
        xref="x",
        yref="y",
        text="Prediction",
        showarrow=False,
        font=dict(
            size=12,
            color="Red"
        ),
        align="center",
        xanchor="left"
    )
)

# Show the plot
fig.show()

Since RCP deals with CO2 emission and the 8.5 scenario has a higher emission prediction than the 4.5 scenario, the annual temperature increase of rcp8.5 is more rapid than rcp4.5 although both are increasing.


In [ ]:
#| code-summary: Temperature comparison (Vegetation)


# Filter the DataFrame for 'RCP' values 'historical' and 4.5
filtered_data = df_con[df_con['RCP'].isin(['historical', 4.5])].groupby(['year','veg'])['T_Annual'].mean().reset_index()

# Create the line plot
fig = px.line(
    data_frame=filtered_data,
    x='year',
    y='T_Annual',
    color='veg',  # This will create lines for each unique value in 'veg' and color them differently
    title='<b>Annual Temperature by Vegetation Type</b>',
    labels={'T_Annual': 'Annual Temperature'}
)
fig.update_layout(title={'x':0.5})


# Add a vertical line at year 2021
fig.add_shape(
    dict(
        type='line',
        x0=2021,
        y0=filtered_data['T_Annual'].min()/1.1,
        x1=2021,
        y1=filtered_data['T_Annual'].max()*1.1,
        line=dict(
            color="Red",
            width=2,
            dash="dash",
        ),
    )
)

fig.add_annotation(
    dict(
        x=2021,  # Position the text to the right of the line
        y=filtered_data['T_Annual'].max(),  # Position the text at the middle of the y-axis
        xref="x",
        yref="y",
        text="Prediction",
        showarrow=False,
        font=dict(
            size=12,
            color="Red"
        ),
        align="center",
        xanchor="left"
    )
)

# Show the plot
fig.show()

The vegetations seem to follow exactly the same pattern of prediciton with a fixed difference between each other. This may mean that when calculating predictions based on scenarios, the algorithm was modeled so that the mean of the vegetations were always a given distance apart from each other. Because of this limitation of the algorithm, **it is unncessary to compare vegetations from each other.** We will always get the same difference.


In [ ]:
#| code-summary: Temperature comparison (scenario)

# Assuming df_orig is your DataFrame and it has been filtered to exclude 'Hist' from 'TimePeriod'
df_filtered = df_orig[df_orig['TimePeriod'] != 'Hist']

# Calculate the median of 'T_Annual' for each 'scenario'
medians = df_filtered.groupby('scenario')['T_Annual'].median().reset_index()

# Sort the median values
medians = medians.sort_values('T_Annual')

# Merge sorted median DataFrame back to the filtered DataFrame
df_sorted = pd.merge(medians['scenario'], df_filtered, on='scenario', how='left')

# Creating a boxplot with Plotly Express using the sorted DataFrame
fig = px.box(df_sorted, x='scenario', y='T_Annual', color='RCP')

# Rotating x-axis labels
fig.update_layout(
    xaxis_tickangle=-90,
    title={
        'text': "<b>Annual Temperature by Scenario</b>",
        'x':0.5,
        'xanchor': 'center'
    }
)
# Displaying the plot
fig.show()

Since we already know that RCP plays a big role in how the algorithm predicts the temperature, We will divide the scenarios into 4.5 scenarios and 8.5 scenarios to see if there is a significant difference. By examining the plot we now know that RCP 4.5 correspons to scenario 22~41 and RCP 8.5 correspons to scenario 42~61. There are cases where 4.5 scenarios had higher temperatures than 8.5 scenarios, but since RCP acts as the first drill down layer of the dataset, we can use the scenario as the second drilldown of the dataset.

For our dataset analysis, we will be comparing the maximum and minimum scenario for each RCP group to analyze what features affect temperature the most. That is comparing scenario 37 to scenario 40 for RCP 4.5 scenarios, and comparing scenario 58 to scenario 60 to do the same for RCP 8.5.

## Statistical Significance
**Is there a significant difference between different scenarios?**

Before we start analyzing our dataset, one final step we want to take is proving the statistical significance in the different scenarios we plan on comparing.

The three comparisons we plan on making are as follows:

1. RCP 8.5(High) vs RCP 4.5(Low)
2. RCP 4.5 : Scenario 37(High) vs Scenario 40(Low)
2. RCP 8.5 : Scenario 60(High) vs Scenario 58(Low)


In [ ]:
#| code-summary: T-test for RCP 4.5 and 8.8

 
# Creating data groups
data_before = df_orig[(df_orig['RCP'] == 8.5) & (df_orig['TimePeriod'] != 'Hist')]['T_Annual']
data_after = df_orig[(df_orig['RCP'] == 4.5) & (df_orig['TimePeriod'] != 'Hist')]['T_Annual']
 
# Conducting two-sample ttest
result = pg.ttest(data_before,
                  data_after,
                  correction=True)
 

# # Print the result
# print("t value : ",result['T'][0])
# print("95% Confidence Interval : ", result['CI95%'][0])
# print("p-value : ", result['p-val'][0])

**T-test for RCP 4.5 and 8.8**

Result                          | Value          |
--------------------------------|----------------|
**t-value**                     | 232.998        |
**95% Confidence Interval**     | [1.25 1.27]    |
**p-Value**                     | 0.000          |      


In [ ]:
#| code-summary: T-test for Scenario 40 vs 37

 
# Creating data groups
data_before = df_orig[df_orig['scenario'] == 'sc40']['T_Annual']
data_after = df_orig[df_orig['scenario'] == 'sc37']['T_Annual']
 
# Conducting two-sample ttest
result = pg.ttest(data_before,
                  data_after,
                  correction=True)
 
# # Print the result
# # Print the result
# print("t value : ",result['T'][0])
# print("95% Confidence Interval : ", result['CI95%'][0])
# print("p-value : ", result['p-val'][0])

**T-test for Scenario 40 vs 37**

Result                          | Value          |
--------------------------------|----------------|
**t-value**                     | -157.977       |
**95% Confidence Interval**     | [-2.51 -2.45]  |
**p-Value**                     | 0.000          |        


In [ ]:
#| code-summary: T-test for Scenario 60 vs 58

# Creating data groups
data_before = df_orig[df_orig['scenario'] == 'sc60']['T_Annual']
data_after = df_orig[df_orig['scenario'] == 'sc58']['T_Annual']
 
# Conducting two-sample ttest
result = pg.ttest(data_before,
                  data_after,
                  correction=True)
 
# # Print the result
# print("t value : ",result['T'][0])
# print("95% Confidence Interval : ", result['CI95%'][0])
# print("p-value : ", result['p-val'][0])

**T-test for Scenario 60 vs 58**

Result                          | Value          |
--------------------------------|----------------|
**t-value**                     | -125.742       |
**95% Confidence Interval**     | [-3.61 -3.5 ]  |
**p-Value**                     | 0.000          | 


# Conclusion
Now that we've proved that the differenc between RCP scenarios, and the highest and lowest scenario for each RCP are all statistically significant, lets dive deeper into the dataset to construct visualizations to hypothesize features that have correlations to the predicted temperature!
